In [1]:
import numpy as np

In [4]:
import clease
from clease.settings import Concentration
import numpy as np
from clease.settings import CEBulk
from clease.structgen import NewStructures
from ase.db import connect
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation
from pymatgen.io.cif import CifWriter
import json, os 


In [10]:
def closest_composition(comp, num_atoms):
    # Calculate the total fraction
    total_fraction = sum(comp.values())
    
    # Calculate the number of atoms for each element
    atoms = {element: fraction / total_fraction * num_atoms for element, fraction in comp.items()}
    
    # Round up non-integer atom counts and adjust the first element to balance
    elements = list(atoms.keys())
    for i in range(1, len(elements)):
        if atoms[elements[i]] % 1 != 0:
            atoms[elements[i]] = int(atoms[elements[i]]) + 1
            #num_atoms += 1

    atoms[elements[0]] = num_atoms - sum(atoms[elements[i]] for i in range(1, len(elements)))
    
    # Calculate the actual fractions
    actual_fractions = {element: num / num_atoms for element, num in atoms.items()}
    
    return atoms, actual_fractions

def create_random_supercells(composition,alat,supercell_size,db_name,num_structures):
    A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
    b_eq = composition.values()
    conc = Concentration(basis_elements=[list(composition.keys())],
                         A_eq = A_eq,
                         b_eq = b_eq)
    settings = CEBulk(crystalstructure='bcc',
                      a=alat,
                      size=[supercell_size,supercell_size,supercell_size],
                      concentration=conc,
                      db_name=db_name,
                      max_cluster_dia=[6.0, 4.5, 4.5])
    


    ns = NewStructures(settings, generation_number=1,
                    struct_per_gen=num_structures)
    ns.generate_random_structures()

def connect_ase_db(db_name):
    from ase.db import connect
    atoms = connect(db_name)
    print(len(atoms))
    for idx in range(len(atoms)):
        curr_struct = atoms.get(id=idx+1).toatoms()
        print(curr_struct)
    return atoms

        
def ase_db_to_pymatgen(db_name,output_file=None):

    data = {}
    Adaptor = AseAtomsAdaptor()
    atoms = connect(db_name) 

    for idx in range(2,len(atoms)+1):
        
        curr_struct = atoms.get(id=idx).toatoms()
        
        curr_pymatgen = Adaptor.get_structure(curr_struct)
        
        
        trans = PerturbStructureTransformation(distance=0.1, min_distance=0.01)
        distorted_struct = trans.apply_transformation(curr_pymatgen)
        
        data = {f'{idx}':distorted_struct.as_dict()}
        
        if output_file is not None:
            with open(output_file, 'w') as f:
                json.dump(data, f,)
        else:
            return data
        

In [11]:
comp = {'V' : 0.92 , 'Cr' : 0.04, 'Ti' : 0.04}
num_atoms = 64
atoms, actual_fractions = closest_composition(comp, num_atoms)

print("Number of atoms for each element:", atoms)
print("Actual fractions for each element:", actual_fractions)

Number of atoms for each element: {'V': 58, 'Cr': 3, 'Ti': 3}
Actual fractions for each element: {'V': 0.90625, 'Cr': 0.046875, 'Ti': 0.046875}


In [12]:
create_random_supercells(composition=comp,
                         alat = 3.0,
                         supercell_size=4,
                         db_name='v4cr4ti.db',
                         num_structures=30)

TypeError: Concentration.__init__() got an unexpected keyword argument 'db_name'

In [3]:
from clease.settings import Concentration

A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
b_eq = [0.90625,0.046875,0.046875]
conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

In [4]:
from clease.settings import CEBulk
settings = CEBulk(crystalstructure='bcc',
                  a=3.0,
                  size=[4,4,4],
                  concentration=conc,
                  db_name="V_Cr_Ti_random_supercells_Nov_29_2023.db",
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [5]:
from clease.structgen import NewStructures


ns = NewStructures(settings, generation_number=1,
                   struct_per_gen=10)
ns.generate_random_structures()


In [6]:
A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
b_eq = [0.65625,0.171875,0.171875]
conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

In [7]:
from clease.settings import CEBulk
settings = CEBulk(crystalstructure='bcc',
                  a=3.0,
                  size=[4,4,4],
                  concentration=conc,
                  db_name="V_Cr_Ti_random_supercells_Nov_29_2023.db",
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [8]:

ns = NewStructures(settings, generation_number=2,
                   struct_per_gen=10)
ns.generate_random_structures()

In [9]:
A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
b_eq = [0.4375,0.28125,0.28125]
conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

In [10]:
from clease.settings import CEBulk
settings = CEBulk(crystalstructure='bcc',
                  a=3.0,
                  size=[4,4,4],
                  concentration=conc,
                  db_name="V_Cr_Ti_random_supercells_Nov_29_2023.db",
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [11]:

ns = NewStructures(settings, generation_number=3,
                   struct_per_gen=10)
ns.generate_random_structures()

In [12]:
from ase.db import connect
atoms = connect('V_Cr_Ti_random_supercells_Nov_29_2023.db')
print(len(atoms))
for idx in range(len(atoms)):
    curr_struct = atoms.get(id=idx+1).toatoms()
    print(curr_struct)

31
Atoms(symbols='V', pbc=True, cell=[[-1.5, 1.5, 1.5], [1.5, -1.5, 1.5], [1.5, 1.5, -1.5]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=Tr

In [13]:
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation
from pymatgen.io.cif import CifWriter

Adaptor = AseAtomsAdaptor()

for idx in range(2,len(atoms)+1):
    
    curr_struct = atoms.get(id=idx).toatoms()
    
    curr_pymatgen = Adaptor.get_structure(curr_struct)
    
    
    trans = PerturbStructureTransformation(distance=0.1, min_distance=0.01)
    distorted_struct = trans.apply_transformation(curr_pymatgen)
    
    writer = CifWriter(distorted_struct)
    writer.write_file("VCrTi_Supercells_Nov_29_2023/Struct_num"+str(idx)+".cif")

In [14]:
print(len(atoms))

31
